In [7]:
from ctu_crs import CRS97
import numpy as np
from numpy.typing import NDArray
import time
import json
import cv2
import PoseComposer
import camera
import puzzle

In [2]:
robot = CRS97()

Firmware version : MARS8 v1.0 build Pi Oct  2 2017 11:06:45


In [29]:
robot.initialize(home=False)

In [75]:
robot.soft_home()

In [8]:
robot.release()

In [27]:
def move_robot_to(position):
    q0 = robot.q_home
    ik_sols = robot.ik(position)
    assert len(ik_sols) > 0
    closest_solution = min(ik_sols, key=lambda q: np.linalg.norm(q - q0))
    robot.move_to_q(closest_solution)
    robot.wait_for_motion_stop()

In [23]:
cam = camera.Camera()
img = robot.grab_image()
midpoint, R = cam.get_mid_points(img)
centerInRobotCoords = cam.cameraToRobot(midpoint)

[[1]
 [2]]
-3.097862650119012 -0.06488171687553708 0.5932230215646868


In [71]:
centerInRobotCoords[2] = centerInRobotCoords[2] + 0.2

In [72]:
arm_vector = np.array([-1, -1, 0])
normal_vector = np.array([0, 0, 1])
Pose = PoseComposer.make_se3_matrix(centerInRobotCoords, arm_vector, normal_vector)

In [73]:
Pose

array([[-0.70710678, -0.70710678,  0.        ,  0.35135395],
       [-0.70710678,  0.70710678,  0.        ,  0.02572125],
       [ 0.        ,  0.        , -1.        ,  0.26468476],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [74]:
move_robot_to(Pose)